In [19]:
import pandas as pd
import numpy as np
import json
import re
from typing import Dict, List, Tuple

def norm_label(label: str) -> str:

    label = label.strip().lower()
    
    # Remove apostrophes (all types)
    label = re.sub(r"[''']", "", label)
    
    # Replace special characters with spaces (except underscores/hyphens)
    label = re.sub(r"[^\w\s-]", " ", label)
    
    # Replace spaces and hyphens with underscores
    label = re.sub(r"[\s-]+", "_", label)
    
    # Remove consecutive underscores
    label = re.sub(r"_+", "_", label)
    
    # Strip leading/trailing underscores
    label = label.strip("_")
    
    return label

def build_index_map_from_json(json_path: str) -> Dict[str, List[str]]:

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    index_map = {}
    missing_index_keys = []
    
    for category in data.get('categories', []):
        category_name = category.get('category_name', 'UNKNOWN')
        
        for sub_group in category.get('sub_groups', []):
            index_key = sub_group.get('index_key')
            group_name = sub_group.get('group_name', 'UNKNOWN')
            
            if not index_key:
                print(f"  Warning: No 'index_key' for '{group_name}' in {category_name}")
                index_key = norm_label(group_name) + "_index"
                missing_index_keys.append(group_name)
            
            # Collect and normalize keywords
            keywords = []
            for query in sub_group.get('queries', []):
                label = query.get('label', '')
                if label:
                    normalized = norm_label(label)
                    keywords.append(normalized)
            
            if keywords:
                index_map[index_key] = keywords
            else:
                print(f"  Warning: No keywords for index '{index_key}'")
    
    if missing_index_keys:
        print(f"\n  {len(missing_index_keys)} groups missing 'index_key': {missing_index_keys}")
    
    return index_map

def build_index_matrix(
    df: pd.DataFrame,
    index_map: Dict[str, List[str]],
    country: str = "Country",
    agg: str = "median",
    min_present: int = 1
) -> Tuple[pd.DataFrame, pd.DataFrame]:

    # Validation
    if country not in df.columns:
        raise ValueError(f"'{country}' column not found in DataFrame. Available: {df.columns.tolist()}")
    
    # Keep non-feature columns
    keep_cols = [country]
    if "date" in df.columns:
        keep_cols.append("date")
    
    # Convert feature columns to numeric
    feature_cols = [c for c in df.columns if c not in keep_cols]
    X_num = df[feature_cols].apply(pd.to_numeric, errors="coerce")
    
    # Initialize output with preserved columns
    out = df[keep_cols].copy()
    
    # Build indices
    report_rows = []
    
    for idx_name, keywords in index_map.items():
        # Find present and missing keywords
        present = [k for k in keywords if k in X_num.columns]
        missing = [k for k in keywords if k not in X_num.columns]
        
        # Record stats
        report_rows.append({
            "index": idx_name,
            "n_keywords": len(keywords),
            "n_present": len(present),
            "n_missing": len(missing),
            "coverage": len(present) / len(keywords) if keywords else 0,
            "missing_keywords": missing[:5]  # First 5 for brevity
        })
        
        # Skip if insufficient keywords
        if len(present) < min_present:
            out[idx_name] = np.nan
            print(f" Skipping '{idx_name}': only {len(present)}/{len(keywords)} keywords present")
            continue
        
        # Aggregate
        if agg == "median":
            out[idx_name] = X_num[present].median(axis=1, skipna=True)
        elif agg == "mean":
            out[idx_name] = X_num[present].mean(axis=1, skipna=True)
        else:
            raise ValueError(f"Invalid agg method: '{agg}'. Use 'median' or 'mean'")
    
    # Create report DataFrame
    index_report = pd.DataFrame(report_rows)
    index_report = index_report.sort_values('coverage')
    
    return out, index_report

def validate_index_map(
    index_map: Dict[str, List[str]], 
    df_columns: List[str],
    min_coverage: float = 0.5
) -> None:
    print("="*70)
    print("INDEX MAP VALIDATION")
    print("="*70)
    
    col_set = set(df_columns)
    total_keywords = sum(len(keywords) for keywords in index_map.values())
    total_present = 0
    low_coverage_indices = []
    
    for idx_name, keywords in index_map.items():
        present = [k for k in keywords if k in col_set]
        missing = [k for k in keywords if k not in col_set]
        coverage = len(present) / len(keywords)
        
        total_present += len(present)
        
        if coverage < min_coverage:
            low_coverage_indices.append((idx_name, coverage, missing))
    
    # Summary
    print(f"Total Keywords: {total_keywords}")
    print(f"Present: {total_present} ({total_present/total_keywords:.1%})")
    print(f"Missing: {total_keywords - total_present}")
    
    # Warnings
    if low_coverage_indices:
        print(f"\n{len(low_coverage_indices)} indices below {min_coverage:.0%} coverage:")
        for idx_name, cov, missing in low_coverage_indices:
            print(f"   • {idx_name}: {cov:.1%}")
            print(f"     Missing: {', '.join(missing[:3])}")
            if len(missing) > 3:
                print(f"     ... and {len(missing)-3} more")
    else:
        print(f"\n All indices have ≥{min_coverage:.0%} coverage!")
    
    print("="*70)
INDEX_MAP = build_index_map_from_json('data/raw/keywords_FINAL_2025.json')



In [20]:
df = pd.read_csv("data/processed/labeled_time_series.csv")
print([c for c in df.columns if c.lower() in ["country", "geo"]])
print(df.columns[:20])
print("Country in columns?", "Country" in df.columns)


['Country']
Index(['date', 'Country', 'bitcoin', 'inflation', 'ferrari', 'patek_philippe',
       'rolex', 'lamborghini', 'louis_vuitton', 'nuclear_energy', 'recycling',
       'tesla', 'veganism', 'global_warming', 'solar_power', 'freelance',
       'corruption', 'tinder', 'phishing', 'cartel'],
      dtype='object')
Country in columns? True


In [21]:
df_indexed, index_report = build_index_matrix(df, INDEX_MAP, country="Country", agg="median", min_present=1)

print("Indexed shape:", df_indexed.shape)
display(df_indexed.head())
display(index_report.tail(10))


 Skipping 'social_identity_politics_index': only 0/4 keywords present
Indexed shape: (11060, 32)


,Country,date,finance_crypto_speculation_index,finance_real_estate_pressure_index,finance_macro_pressure_index,luxury_status_consumption_index,luxury_aesthetic_body_index,luxury_elite_mobility_index,crime_gun_violence_index,crime_organized_crime_index,...,tech_digital_intelligence_index,tech_security_privacy_index,entertainment_streaming_consumption_index,entertainment_short_video_index,health_mental_health_index,health_healthcare_access_index,health_addiction_recovery_index,health_sleep_crisis_index,sports_body_image_index,sports_elite_spectator_sports_index
0,US,2021-12-26,8.0,34.5,2.0,17.0,17.5,5.0,0.5,60.0,...,0.0,0.5,55.0,69.0,5.0,7.0,0.0,0.0,2.0,30.0
1,US,2022-01-02,10.0,44.0,4.0,14.0,17.5,5.0,0.5,70.0,...,0.0,1.0,53.5,65.0,7.0,12.0,0.0,0.0,4.0,19.0
2,US,2022-01-09,10.0,44.5,4.0,12.5,17.5,5.0,0.5,72.0,...,0.0,1.0,55.5,65.0,7.0,10.5,0.0,0.0,4.0,18.0
3,US,2022-01-16,10.0,43.5,5.0,12.5,17.5,5.0,0.5,68.0,...,1.0,1.0,53.5,59.0,7.0,10.5,0.0,0.0,4.0,13.0
4,US,2022-01-23,10.5,48.0,5.0,16.5,17.5,4.0,0.5,72.0,...,1.0,1.5,52.5,64.0,8.0,11.5,0.0,0.0,4.0,15.0


,index,n_keywords,n_present,n_missing,coverage,missing_keywords
13,social_corruption_index,3,2,1,0.666667,[offshore_account]
20,tech_digital_intelligence_index,10,7,3,0.700000,"[machine_learning, quantum_computer, quantum_s..."
12,social_work_culture_index,4,3,1,0.750000,[work_from_anywhere]
11,social_migration_pressure_index,4,3,1,0.750000,[asylum_seeker]
26,health_addiction_recovery_index,5,4,1,0.800000,[fentanyl_crisis]
18,environment_energy_transition_index,10,8,2,0.800000,"[electric_car, green_technology]"
8,crime_sexual_violence_index,6,5,1,0.833333,[metoo]
21,tech_security_privacy_index,7,6,1,0.857143,[privacy]
10,crime_violent_crime_index,4,4,0,1.000000,[]
27,health_sleep_crisis_index,5,5,0,1.000000,[]


In [22]:
print("Old feature count:", df.shape[1]-1)
print("New index count:", df_indexed.shape[1]-1)


Old feature count: 108
New index count: 31


In [23]:
index_cols = [c for c in df_indexed.columns if c not in ["date", "Country"]]

country_index_matrix_median = (
    df_indexed
    .groupby("Country")[index_cols]
    .median()
    .reset_index()
)


In [24]:
df_indexed.to_csv("data/processed/country_index_matrix_median.csv", index=False)
index_report.to_csv("data/processed/index_build_report.csv", index=False)
print("Saved outputs to data/processed/")


Saved outputs to data/processed/


In [25]:
import pandas as pd

df = pd.read_csv("data/processed/labeled_time_series.csv")

# standardize: geo -> Country
if "geo" in df.columns and "Country" not in df.columns:
    df = df.rename(columns={"geo":"Country"})

cols = set(df.columns)

report = []
for idx, keys in INDEX_MAP.items():
    present = [k for k in keys if k in cols]
    missing = [k for k in keys if k not in cols]
    report.append((idx, len(keys), len(present), len(missing), missing[:8]))

report_df = pd.DataFrame(report, columns=["index","n_keywords","n_present","n_missing","missing_sample"])
report_df["coverage"] = report_df["n_present"]/report_df["n_keywords"]

display(report_df.sort_values("coverage"))



,index,n_keywords,n_present,n_missing,missing_sample,coverage
15,social_identity_politics_index,4,0,4,"[lgbt_social_movements, trans_rights, gender_a...",0.000000
7,crime_organized_crime_index,3,1,2,"[mafia, gang_violence]",0.333333
23,entertainment_short_video_index,3,1,2,"[shorts, reels]",0.333333
16,social_vegan_lifestyle_index,3,1,2,"[plant_based_diet, tofu]",0.333333
17,environment_climate_concern_index,5,2,3,"[carbon_footprint, carbon_emissions, climate_p...",0.400000
22,entertainment_streaming_consumption_index,5,2,3,"[disney_plus, pirated_movies, streaming_service]",0.400000
6,crime_gun_violence_index,5,2,3,"[gun_violence, open_carry, concealed_carry]",0.400000
1,finance_real_estate_pressure_index,5,2,3,"[real_estate, housing_market, housing_bubble]",0.400000
5,luxury_elite_mobility_index,7,3,4,"[bugatti, porsche_911, private_jet, gulfstream]",0.428571
24,health_mental_health_index,7,3,4,"[anxiety_disorder, mental_disorder, mental_wel...",0.428571


In [26]:
# select index columns and calculate variances 
index_cols = [c for c in df_indexed.columns if c not in ["date", "Country"]]
variances = df_indexed[index_cols].var().sort_values(ascending=False)

# df to visualize
variance_report = pd.DataFrame({'Index': variances.index, 'Variance': variances.values})
print(variance_report)

                                        Index    Variance
0                 luxury_elite_mobility_index  459.698655
1                 crime_organized_crime_index  335.777489
2                 social_dating_culture_index  299.501401
3                social_vegan_lifestyle_index  279.058594
4             luxury_status_consumption_index  276.356261
5                  health_mental_health_index  246.943897
6                      crime_cybercrime_index  218.783137
7             entertainment_short_video_index  194.919088
8         sports_elite_spectator_sports_index  174.465922
9          finance_real_estate_pressure_index  172.957960
10                  social_work_culture_index  157.346218
11                    social_corruption_index  124.139506
12                environment_pollution_index  100.757455
13                luxury_aesthetic_body_index   91.159740
14             health_healthcare_access_index   86.590767
15                crime_sexual_violence_index   83.217508
16            

In [27]:
# 1. Load the index matrix
df_indices = pd.read_csv('data/processed/country_index_matrix_median.csv')

# 2. Select numerical columns (excluding 'Country' and 'date')
numeric_cols = [c for c in df_indices.columns if c not in ['Country', 'date']]

# 3. Calculate variances and determine the threshold (0.1)
# Features with variance less than the threshold are considered "not informative"
variances = df_indices[numeric_cols].var()
threshold = 0.1

low_variance_features = variances[variances < threshold].index.tolist()
high_variance_features = [c for c in df_indices.columns if c not in low_variance_features]

# 4. Remove low-variance features
df_high_var = df_indices[high_variance_features]

# 5. Save the new file
df_high_var.to_csv('data/processed/country_index_matrix_HIGH_VARIANCE.csv', index=False)

print(f"Threshold: {threshold}")
print(f"Number of Removed Features: {len(low_variance_features)}")
print(f"Number of Remaining Features (excluding 'Country' and 'date'): {len(high_variance_features) - 2}")
print("\nInsignificant Features:")
for f in low_variance_features:
    print(f"- {f}")

Threshold: 0.1
Number of Removed Features: 0
Number of Remaining Features (excluding 'Country' and 'date'): 30

Insignificant Features:


## Variance Analysis – Key Takeaways

- Cultural differences between countries are primarily driven by **consumption patterns, social relationships, crime perception, and work–lifestyle dynamics**.

- Topics such as **health, sports, artificial intelligence, quantum computing, and brain–computer interfaces** exhibit **uniform interest across countries**, indicating global hype phenomena.

- These globally popular topics contribute **little to clustering performance**, as they display **minimal country-specific variance**.

- Low-variance features behave as expected.  
  For example, the **gun violence index (variance = 0.046)** shows a strong outlier effect.

- In gun violence–related interest, the **United States stands out sharply**, while most other countries exhibit **consistently low and similar levels of interest**, resulting in low overall variance.

- **Very low-variance features were removed** to prevent them from diluting the clustering structure.

- The remaining features provide a **more informative representation of cross-country differences**, leading to more meaningful clustering results.
